Pablo Jimeno - UPV/EHU 2016

In [ ]:
import os
import pyfits
import numpy as np

from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

H0_Planck = 67.74
Om_b_Planck = 0.022300 / (H0_Planck/100.)**2
Om_dm_Planck = 0.308889 - Om_b_Planck
Om_m_Planck = 0.308889
#sigma_8_Planck = 0.8159

from astropy.cosmology import Planck13

default_time = Time('2017-01-01', format='isot', out_subfmt='date', scale='utc').jd # = 2457754.5


##################################################################################################################
### Binning utilities:
##################################################################################################################

#-------------------------------------------------------------------
def automatic_binning(R_vals, z_vals):
       
    r_min = 20.
    r_max = 300.
    z_min = 0.100
    z_max = 0.600
    
    # Automatic binning:
    #sample_rich = raw_redm['rich']
    #sample_z    = raw_redm['photo_z']
    # r_vals = [np.array_split(np.sort(sample_rich), nr_bins)[x][-1] for x in range(nr_bins-1)]
    # z_vals = [np.array_split(np.sort(sample_z), nz_bins)[x][-1] for x in range(nz_bins-1)] 
    # r_vals.insert(0, r_min)
    # r_vals.append(r_max)
    # z_vals.insert(0, z_min)
    # z_vals.append(z_max)
    # R bins: 2=[33.3],  3=[28.0, 40.5],   4=[25.8, 33.3, 45.7],    5=[24.5, 30.0, 37.3, 49.6]
    # z bins: 2=[0.372], 3=[0.332, 0.415], 4=[0.294, 0.372, 0.437], 5=[0.268, 0.350, 0.398, 0.450]
        
    ## Select subsample:  [rich_min, rich_max, z_min, z_max]
    subsample_bins = [[R_vals[i], R_vals[i+1], z_vals[j], z_vals[j+1]]
                          for i in range(len(R_vals)-1) for j in range(len(z_vals)-1)]
    
    nr_bins = (len(R_vals)-1)
    nz_bins = (len(z_vals)-1)
                    
    print '\nr_min \t r_max \t z_min \t z_max\n' + '-'*50
    for sub_bin in subsample_bins:
        if (sub_bin[0]<sub_bin[1]) or (sub_bin[2]<sub_bin[3]):
            print '%i \t %i \t %.3f \t %.3f' % tuple(sub_bin)
    print '\n'+'*'*50
    
    return subsample_bins, nr_bins, nz_bins   


##################################################################################################################
### redMaPPer utilities:
##################################################################################################################


def import_redMaPPer(filedir):
    """
    Function to read and import fits catalogs.
    Output is dict with keys: 
    id, ra[0:5], dec[0:5], spec_z, photo_z, photo_z_err, rich, rich_err, prob[0:5], s, n_gal
    """
    
    print '\n********************'
    print '\nReading catalogue found in: %s ...' % filedir
        
    raw_catalog = Table.read(filedir, format='fits')
    
    print '\nNumber of raw objects found: %i' % len(raw_catalog)
    print '\nRaw columns:\n\n%s' % raw_catalog.colnames
    
    catalog = {}
    
    catalog['id'] = raw_catalog['ID'].data
    catalog['ra'] = raw_catalog['RA_CEN'].data
    catalog['dec'] = raw_catalog['DEC_CEN'].data
    catalog['spec_z'] = raw_catalog['Z_SDSS'].data
    catalog['photo_z'] = raw_catalog['Z_LAMBDA'].data
    catalog['photo_z_err'] = raw_catalog['Z_LAMBDA_ERR'].data
    catalog['rich'] = raw_catalog['LAMBDA'].data
    catalog['rich_err'] = raw_catalog['LAMBDA_ERR'].data
    catalog['s'] = raw_catalog['S'].data
    catalog['n_gal'] = catalog['rich']/catalog['s']
    catalog['prob'] = raw_catalog['P_CEN'].data
        
    print '\nCreated catalog with keys:\n\n/',    
    for key, value in catalog.iteritems():
        print key , '/',
    print '\n'    
    
    return catalog



def weighted_redshift(sample):
       
    z_mask = sample['spec_z'] > 0.
    z_mask_any = z_mask.any(axis=1)

    prob = ma.masked_array(sample['prob'], mask=~z_mask)

    dot_z = prob*sample['spec_z']
    sum_z = dot_z.sum(axis=1)

    sum_prob = prob.sum(axis=1)

    z_avg = (sum_z/sum_prob).data[z_mask_any]

    sample_z_avg = np.zeros(len(sample['id']))

    sample_z_avg[~z_mask_any] = -1.
    sample_z_avg[z_mask_any]  = z_avg

    return sample_z_avg



def create_subsample(sample, subkey='z', min_val=0., max_val=0.9999):
    """
    Create subsample from catalogue.
    Output is equivalent to sample, with subkey mask applied.
    """
    
    subsample = {}
    
    if subkey in sample.keys():
        mask = (sample[subkey] >= min_val) & (sample[subkey] < max_val)
    else:
        print '***ERROR: subkey %s not found in catalogue.' % subkey
    
    for key in sample.keys():
        subsample[key] = sample[key][mask]
    
    return subsample


##################################################################################################################
### DR12 utilities:
##################################################################################################################

    
def import_DR12(filedir, sample='SDSS'):
    """
    Imports SDSS DR12 sample.
    Four possible samples: SDSS, BOSS; CMASS, Legacy.
    Output is dict with keys: 
    ra, dec, z
    """
        
    print '\nReading catalog found in: %s ...' % filedir

    data = pyfits.open(filedir)
        
    raw = data[1].data
    data.close()
    
    catalog = {}
     
    global_mask = (((raw.field('PLATEQUALITY') == 'good') | (raw['PLATEQUALITY'] == 'marginal')) &
                  (raw['CLASS'] == 'GALAXY'))
    
    boss_mask =  ((raw.field('SPECBOSS') == 1) &                 
                 (raw.field('Z_NOQSO') >= 0.00001) & 
                 (raw.field('Z_NOQSO') < 1.) & 
                 (raw.field('Z_ERR_NOQSO') < 0.001) &
                 ((raw.field('ZWARNING_NOQSO') == 0) | (raw.field('ZWARNING_NOQSO') == 16)))
    
    cmass_mask = ((raw.field('CHUNK') != "boss1") &
                  (raw.field('CHUNK') != "boss2") &
                  (raw.field('BOSS_TARGET1') & 2 != 0))
    
    lega_mask =  ((raw.field('SPECLEGACY') == 1) &
                 (raw.field('Z') >= 0.00001) & 
                 (raw.field('Z') < 1.) & 
                 (raw.field('Z_ERR') < 0.001) & 
                 ((raw.field('ZWARNING') == 0) | (raw.field('ZWARNING') == 16)))

    sdss_mask =  (raw.field('SPECPRIMARY') == 1)
    
    cat_raw = Table(raw, copy=False)
    
    print '\nNumber of raw objects found: %i' % len(cat_raw)
    print '\nColumns:\n\n', cat_raw.colnames
            
    if sample=='SDSS':
        
        dr12_mask = (global_mask & ((sdss_mask & boss_mask) | (sdss_mask & lega_mask)))
        
        n_gal = len(cat_raw['SPECPRIMARY'][dr12_mask])
        catalog['z'] = np.zeros(n_gal)
        
        z_v2 = cat_raw['Z_NOQSO'][dr12_mask].data
        z_v1 = cat_raw['Z'][dr12_mask].data
        
        for z in range(n_gal):
            if z_v2[z]==0.:
                catalog['z'][z] = z_v1[z]
            elif z_v2[z]!=0.:
                catalog['z'][z] = z_v2[z]
                
    elif sample=='BOSS':
        
        dr12_mask = global_mask & boss_mask
        catalog['z'] = cat_raw['Z_NOQSO'][dr12_mask].data
        
    elif sample=='CMASS':
        
        dr12_mask = global_mask & boss_mask & cmass_mask
        catalog['z'] = cat_raw['Z_NOQSO'][dr12_mask].data
        
    elif sample=='Legacy':
        
        dr12_mask = global_mask & lega_mask
        catalog['z'] = cat_raw['Z'][dr12_mask].data
        
    catalog['ra'] = cat_raw['PLUG_RA'][dr12_mask].data
    catalog['dec'] = cat_raw['PLUG_DEC'][dr12_mask].data
        
    print '\nCreated catalog with keys:\n'    
    for key, value in catalog.iteritems():
        print key, ' --> ', value
        
    print '\nNumber of objects in new catalog: %i' % len(catalog['z'])
    
    return catalog    